In [17]:
import requests
import json
import pandas as pd
import math
from datetime import datetime, timedelta
from gateway.populous import PopulousClient


client = requests.session()

headers = {}

data = {}

HOST = "https://api.surveymonkey.net"
SURVEY_LIST_ENDPOINT = "/v3/surveys"

uri = "%s%s" % (HOST, SURVEY_LIST_ENDPOINT)

response = client.get(uri, headers=headers)

response_json = response.json()
# survey_list = response_json["data"]["surveys"]

print(response_json)

{u'per_page': 50, u'total': 2, u'data': [{u'href': u'https://api.surveymonkey.net/v3/surveys/297202682', u'nickname': u'', u'id': u'297202682', u'title': u'Test\xa0of Uber Community Guidelines Quiz'}, {u'href': u'https://api.surveymonkey.net/v3/surveys/160741385', u'nickname': u'', u'id': u'160741385', u'title': u'Uber Community Guidelines Quiz'}], u'page': 1, u'links': {u'self': u'https://api.surveymonkey.com/v3/surveys?page=1&per_page=50'}}


In [18]:
survey_id = 160741385
collector_id = 219458600

In [19]:
def UtcNow():
    now = datetime.now() + timedelta(minutes=-100*30*24*60)
    return now

UtcNow()

def UtcLat():
    lat = datetime.now() + timedelta(minutes=-10*30*24*60) 
    return lat

UtcLat()

datetime.datetime(2020, 2, 19, 7, 38, 14, 886678)

In [20]:
url= 'https://api.surveymonkey.com/v3/collectors/{}/responses/bulk?simple=true&start_created_at={}&end_created_at={}'.format(collector_id, UtcNow(),UtcLat())

In [21]:
x=client.get(url, headers=headers)
y=x.json()

In [22]:
y['total']

85

In [23]:
per_page=y['per_page']
total = y['total']
tot_pages = total/per_page
tot_pages=math.ceil(tot_pages)

In [8]:
df_fin=pd.DataFrame()

for t in range(1,tot_pages+1):
    url= 'https://api.surveymonkey.com/v3/collectors/{}/responses/bulk?simple=true&start_created_at={}&end_created_at={}&page={}'.format(collector_id, UtcNow(),UtcLat(), t)
    x=client.get(url, headers=headers)
    y=x.json()
    pages = total/per_page
    pages=math.ceil(pages)
    page = t    
    list=[]
    rows_left=total-(page-1)*per_page
    
    if rows_left>=per_page:
        for row in range(per_page):
            i=y['data'][row]
            id = i['id']
            try:
                ip_address = i['ip_address']
            except:
                ip_address = ""
            start_date = i['date_created']
            end_date = i['date_modified']
            try:
                uuid = i['custom_variables']['uuid']
            except:
                uuid = ""
            try:
                incorrect=i['quiz_results']['incorrect']
            except:
                incorrect = ""
            try:
                score=i['quiz_results']['score']
            except:
                score = ""   
            try:
                correct=i['quiz_results']['correct']
            except:
                correct = ""     
#                 q1 = i['pages'][0]['questions'][0]['answers'][0]['simple_text']
#             except:
#                 q1 = ""
#             try:
#                 q2 = i['pages'][2]['questions'][0]['answers'][0]['simple_text']
#             except:
#                 q2 = ""
#             try:
#                 q3 = i['pages'][3]['questions'][0]['answers'][0]['simple_text']
#             except:
#                 q3=""
#             try:
#                 q4 = i['pages'][4]['questions'][0]['answers'][0]['simple_text']
#             except:
#                 q4=""
            list.append([id, ip_address, start_date, end_date, uuid, incorrect, score, correct])

        df=pd.DataFrame(list, columns=["id", "ip_address", "start_date", "end_date", "uuid", "incorrect", "score", "correct"])
        df['start_date'] =df['start_date'].astype('datetime64[s]') 
        df['end_date'] =df['end_date'].astype('datetime64[s]') 
        
    else:
        for row in range(rows_left):
            i=y['data'][row]
            id = i['id']
            try:
                ip_address = i['ip_address']
            except:
                ip_address = ""
            start_date = i['date_created']
            end_date = i['date_modified']
            try:
                uuid = i['custom_variables']['uuid']
            except:
                uuid = ""
            try:
                incorrect=i['quiz_results']['incorrect']
            except:
                incorrect = ""
            try:
                score=i['quiz_results']['score']
            except:
                score = ""   
            try:
                correct=i['quiz_results']['correct']
            except:
                correct = ""         
            
#             try:
#                 q1 = i['pages'][0]['questions'][0]['answers'][0]['simple_text']
#             except:
#                 q1 = ""
#             try:
#                 q2 = i['pages'][2]['questions'][0]['answers'][0]['simple_text']
#             except:
#                 q2 = ""
#             try:
#                 q3 = i['pages'][3]['questions'][0]['answers'][0]['simple_text']
#             except:
#                 q3=""
#             try:
#                 q4 = i['pages'][4]['questions'][0]['answers'][0]['simple_text']
#             except:
#                 q4=""
            list.append([id, ip_address, start_date, end_date, uuid, incorrect, score, correct])

        df=pd.DataFrame(list, columns=["id", "ip_address", "start_date", "end_date", "uuid","incorrect", "score", "correct"])
        df['start_date'] =df['start_date'].astype('datetime64[s]') 
        df['end_date'] =df['end_date'].astype('datetime64[s]') 
    
    df_fin = df_fin.append(df)

In [9]:
df_fin

,id,ip_address,start_date,end_date,uuid,incorrect,score,correct
0,10309501285,64.213.31.50,2018-10-29 02:20:16,2018-10-29 02:20:32,a626855f-464a-485b-8fb9-aec600c3f5d4,2,4,4
1,10354698082,49.178.15.183,2018-11-15 07:48:17,2018-11-15 07:49:42,235fd06b-a134-4871-8ec2-f30f0f6743c4,0,6,6
2,10367724827,210.9.96.210,2018-11-21 12:11:32,2018-11-21 12:12:44,59848be3-dc70-4b1e-8406-7a1aa2dd4aaa,0,6,6
3,10372890467,47.72.25.24,2018-11-24 02:35:42,2018-11-24 02:36:51,8a78b7cc-60f1-4aa5-9e15-8db0d8fb8a3b,1,5,5
4,10373573517,47.72.25.24,2018-11-24 19:00:33,2018-11-24 19:01:15,8a78b7cc-60f1-4aa5-9e15-8db0d8fb8a3b,0,6,6
5,10374771047,49.178.14.77,2018-11-26 00:24:55,2018-11-26 00:47:16,7f0ddaff-b3e0-4638-8570-ef0e96b9a9f6,0,6,6
6,10386794082,58.173.165.199,2018-11-30 08:04:02,2018-11-30 08:05:46,3c57a8d1-a640-4e89-bf13-286d5c1fefde,1,5,5
7,10423383711,43.252.114.80,2018-12-17 07:46:39,2018-12-17 07:47:43,1f593edb-ab96-414a-be74-5e1ca8d0f05d,0,6,6
8,10490940142,1.152.107.243,2019-01-26 07:37:05,2019-01-26 07:39:50,780b3363-017d-4251-9c5b-5d5430f58950,0,6,6
9,10516800489,120.21.210.134,2019-02-07 23:34:58,2019-02-07 23:35:50,e306afe1-7e49-47bd-850d-3a20c9386212,1,5,5


In [10]:
df_pass=df_fin[df_fin.score >=5]

In [11]:
df_fail=df_fin[df_fin.score < 5]

In [12]:
df_fail

,id,ip_address,start_date,end_date,uuid,incorrect,score,correct
0,10309501285,64.213.31.50,2018-10-29 02:20:16,2018-10-29 02:20:32,a626855f-464a-485b-8fb9-aec600c3f5d4,2,4,4
49,10817633410,120.17.95.19,2019-06-24 02:14:45,2019-06-24 02:16:36,b47eecbc-561c-45eb-b424-adbae8680252,2,4,4


In [14]:
tag_name = 'Rider_failed_test'
tag_note= 'Test fail'


In [15]:
rider_tag=df_fail.applymap(str)

In [16]:
for row in rider_tag.index:
    #print(driver_tag['driver_uuid'][row], row)
    PopulousClient.addDriveTag(rider_tag['uuid'][row],tag_name , tag_note)




'a626855f-464a-485b-8fb9-aec600c3f5d4' is Tagged!
'b47eecbc-561c-45eb-b424-adbae8680252' is Tagged!


In [24]:
y

{u'data': [{u'analyze_url': u'https://www.surveymonkey.com/analyze/browse/ES_2FqugEvtudH4MFjrHHqa9bD6yDekKCXgkKTRJ4OJpQ_3D?respondent_id=10309501285',
   u'collection_mode': u'default',
   u'collector_id': u'219458600',
   u'custom_value': u'',
   u'custom_variables': {u'uuid': u'a626855f-464a-485b-8fb9-aec600c3f5d4'},
   u'date_created': u'2018-10-29T02:20:16+00:00',
   u'date_modified': u'2018-10-29T02:20:32+00:00',
   u'edit_url': u'https://www.surveymonkey.com/r/?sm=Y83Q9_2F_2BMMlkaii4Udm6NkE_2BbN5xz_2BkCP0D8_2F1S3fRoHY_2BQ_2FVt7TjsmRcFY_2BlPzD9',
   u'email_address': u'',
   u'first_name': u'',
   u'href': u'https://api.surveymonkey.com/v3/collectors/219458600/responses/10309501285',
   u'id': u'10309501285',
   u'ip_address': u'64.213.31.50',
   u'last_name': u'',
   u'logic_path': {},
   u'metadata': {u'contact': {}},
   u'page_path': [],
   u'pages': [{u'id': u'47771712',
     u'questions': [{u'answers': [{u'choice_id': u'1174664569',
         u'is_correct': False,
         u's